In [4]:
from shapely.geometry import Point, Polygon
import shapefile as shp
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import contextily as ctx
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

df = pd.read_csv("apartments.csv")
pts_col = []

for pt in df[["XCoord", "YCoord"]].itertuples():
    pts_col.append(Point(pt.XCoord, pt.YCoord))
df["geometry"] = pts_col


df = gpd.GeoDataFrame(df)
df = df.set_crs("esri:103412")
df = df.to_crs(epsg=4326)

census_df = pd.read_csv("census_by_block_2010/sex_age.csv")
census_df["GEO_ID"] = census_df["GEO_ID"]
census_df = census_df[["GEO_ID", "NAME", "P012001", "P012004", "P012005", "P012006", "P012028", "P012029", "P012030"]]
census_df = census_df.rename(columns = {"NAME":"Area", "P012001": "Total_people", "P012004": "5-9yrMale", "P012005": "10-14yrMale", "P012006": "15-17yrMale", "P012028": "5-9yrFemale", "P012029": "10-14yrFemale", "P012030": "15-17yrFemale"}, inplace=False)
census_df = census_df[1:]
census_df["Total_people"] = census_df["Total_people"].astype(int)
census_df["5-9yr"] = census_df["5-9yrMale"].astype(int) + census_df["5-9yrFemale"].astype(int)
census_df["10-14yr"] = census_df["10-14yrMale"].astype(int) + census_df["10-14yrFemale"].astype(int)
census_df["15-17yr"] = census_df["15-17yrMale"].astype(int) + census_df["15-17yrFemale"].astype(int)

census_df["Total_K-12_block"] = census_df["5-9yr"] + census_df["10-14yr"] + census_df["15-17yr"]

# census_df[census_df["GEO_ID"] == "1000000US550250114071059"]

dane = gpd.read_file("newdane/newdane.shp")
dane["GEO_ID"] = "1000000US" + dane["GEOID10"]

dane.to_crs(df.crs, inplace=True)

block_with_apartment = gpd.sjoin(df, dane, how="left", op="within")

/opt/anaconda3/envs/CS_HW/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (42,43,104,109,122) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
block_with_apartment.head()

,OBJECTID,Parcel,XRefParcel,Address,DateParcelChanged,PropertyClass,PropertyUse,AssessmentArea,AreaName,MoreThanOneBuild,...,MTFCC10,UR10,UACE10,UATYP10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,GEO_ID
0,1,60905214263,60905203034,2333 Carling Dr Unit 4,2021/04/08 00:00:00+00,Residential,Condominium,5080,Condominium in Res Area 80,NaN,...,G5040,U,53200,U,S,35956,0,+43.0291530,-089.4607171,1000000US550250006003000
1,2,60905214271,60905203034,2337 Carling Dr Unit 1,2021/04/08 00:00:00+00,Residential,Condominium,5080,Condominium in Res Area 80,NaN,...,G5040,U,53200,U,S,35956,0,+43.0291530,-089.4607171,1000000US550250006003000
2,3,60905214289,60905203034,2337 Carling Dr Unit 2,2021/04/08 00:00:00+00,Residential,Condominium,5080,Condominium in Res Area 80,NaN,...,G5040,U,53200,U,S,35956,0,+43.0291530,-089.4607171,1000000US550250006003000
3,4,60905214297,60905203034,2337 Carling Dr Unit 3,2016/02/09 00:00:00+00,Residential,Condominium,6601,Residential Exempt,NaN,...,G5040,U,53200,U,S,35956,0,+43.0291530,-089.4607171,1000000US550250006003000
4,5,60905214304,60905203034,2337 Carling Dr Unit 4,2021/04/08 00:00:00+00,Residential,Condominium,5080,Condominium in Res Area 80,NaN,...,G5040,U,53200,U,S,35956,0,+43.0291530,-089.4607171,1000000US550250006003000


In [18]:
merged = pd.merge(block_with_apartment, census_df, on="GEO_ID", how='inner')
madison_geoid = block_with_apartment["GEO_ID"].drop_duplicates()
census_geoid = census_df["GEO_ID"]
set(madison_geoid) - set(census_geoid)

set()

In [26]:
merged.groupby("GEO_ID").mean().sum()

OBJECTID            1.369191e+08
Parcel              2.391528e+14
XRefParcel          2.391528e+14
AssessmentArea      1.049099e+07
YearBuilt           4.475339e+06
                        ...     
Total_people        2.298350e+05
5-9yr               1.128600e+04
10-14yr             9.993000e+03
15-17yr             5.935000e+03
Total_K-12_block    2.721400e+04
Length: 90, dtype: float64